In [1]:
#ch10_RDD_ALS.ipynb
#Collaborative Filtering- RDD-based API
#http://spark.apache.org/docs/2.2.0/mllib-collaborative-filtering.html#collaborative-filtering

In [1]:
#Collaborative filtering is commonly used for recommender systems.
#spark.mllib uses the alternating least squares (ALS) algorithm

In [2]:
#

In [3]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [4]:
# Load  data
data99RDD = sc.textFile("test99.data")

In [5]:
#parse data
ratingsRDD=data99RDD.map(lambda x:x.split(','))\
.map(lambda x:Rating(int(x[0]), int(x[1]), float(x[2])))

In [6]:
# Build the recommendation model using Alternating Least Squares (ALS)
rank = 10
numIterations = 10
model = ALS.train(ratingsRDD, rank, numIterations)

In [ ]:
# Evaluate the model on training data
testdata = ratingsRDD.map(lambda x: (x[0], x[1]))   #取前兩個欄位資料
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

In [12]:
testdata=ratingsRDD.map(lambda x: (x[0], x[1]))
testdata.collect()

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [18]:
predictions = model.predictAll(testdata).map(lambda r:((r[0], r[1]),r[2]))

In [19]:
predictions.first() #((r[0], r[1]), r[2])

((4, 4), 4.995461549925791)

In [24]:
#將ratingsRDD加工 map後,與predictions join
ratesAndPreds = ratingsRDD.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)

In [25]:
ratesAndPreds.collect() #檢視

[((1, 3), (5.0, 4.996063693942845)),
 ((3, 3), (1.0, 1.000065609810421)),
 ((3, 1), (1.0, 1.000065609810421)),
 ((4, 4), (5.0, 4.995461549925791)),
 ((1, 1), (5.0, 4.996063693942845)),
 ((2, 2), (1.0, 0.9999451228981897)),
 ((4, 2), (5.0, 4.995461549925791)),
 ((2, 4), (1.0, 0.9999451228981897)),
 ((1, 2), (1.0, 0.9999451228981897)),
 ((3, 2), (5.0, 4.995461549925791)),
 ((3, 4), (5.0, 4.995461549925791)),
 ((1, 4), (1.0, 0.9999451228981897)),
 ((2, 3), (5.0, 4.996063693942845)),
 ((2, 1), (5.0, 4.996063693942845)),
 ((4, 3), (1.0, 1.000065609810421)),
 ((4, 1), (1.0, 1.000065609810421))]

In [26]:
#計算MSE
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 9.0248376488e-06


In [27]:
# Save and load model
model.save(sc, "myCollaborativeFilter01") #save model
sameModel = MatrixFactorizationModel.load(sc, "myCollaborativeFilter01")